# Instructions

To run this notebook you will need to:
1. Download Ollam from [here](https://ollama.com/)
2. Go to your terminal and type: `ollama pull llama3` to download llama3 model in your local computer.

## Imports

In [8]:
import pandas as pd
import subprocess
import ast
import re

## Function to extract judge's name

In [9]:
def generate_judge_sentence(text, model="llama3"):
    """
    Generate a sentence identifying the judges in a given court text.

    Parameters
    ----------
    text : str
        The court text to analyze.
    model : str, optional
        The name of the language model to use (default is "llama3").

    Returns
    -------
    str
        A sentence either listing the judges or stating that none are mentioned.
    """
    prompt = f"""
You are a legal assistant. Your task is to identify the names of the judge(s) who presided over the case from the following court text.

Instructions:
- Return a single sentence that starts with "The judges in this case are ..." followed by the judge names.
- If no judge is mentioned, return "No judges are mentioned in the case."
- Do not assume any judges if it is not mentioned.
- Keep your answer in 1 sentence.

Court Text:
{text}
"""
    result = subprocess.run(
        ["ollama", "run", model],
        input=prompt.encode(),
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )
    return result.stdout.decode().strip()


def extract_names_from_judge_sentence(sentence, model="llama3"):
    """
    Extract only the judge names from a sentence.

    Parameters
    ----------
    sentence : str
        A sentence that mentions the judges (e.g., output of `generate_judge_sentence`).
    model : str, optional
        The name of the language model to use (default is "llama3").

    Returns
    -------
    list of str
        A list of judge names with titles removed. Returns an empty list if no names are found.
    """
    prompt = f"""
You are a legal parser. Extract only the judge names from the sentence below.

Instructions:
- Return the names in a valid Python list of strings.
- Do not include any titles like "Judge", "Justice", or "Chief Justice".
- If no names are found, return: []

Sentence:
{sentence}

Output:
"""
    result = subprocess.run(
        ["ollama", "run", model],
        input=prompt.encode(),
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )
    output = result.stdout.decode().strip()

    match = re.search(r"\[.*?\]", output, re.DOTALL)
    if match:
        try:
            return ast.literal_eval(match.group(0))
        except Exception:
            return []
    return []


def extract_judges_from_dataframe(df, text_column="unofficial_text", model="llama3"):
    """
    Extract judge names from a DataFrame containing court texts.

    Parameters
    ----------
    df : pandas.DataFrame
        The DataFrame containing court text data.
    text_column : str, optional
        The column name in `df` that contains the court text (default is "unofficial_text").
    model : str, optional
        The language model to use for generation and parsing (default is "llama3").

    Returns
    -------
    pandas.DataFrame
        A new DataFrame with an additional 'judges' column containing lists of judge names.
    """
    df = df.copy()
    df["judges"] = None

    for idx, row in df.iterrows():
        full_text = row[text_column]
        lines = full_text.splitlines()
        first_50 = "\n".join(lines[:50])

        judge_sentence = generate_judge_sentence(first_50, model=model)
        judge_names = extract_names_from_judge_sentence(judge_sentence, model=model)

        df.at[idx, "judges"] = judge_names
        print(f"Row {idx} processed.")
    
    return df

## Supreme Court Data

In [11]:
SCC = pd.read_csv("../data/processed/SCC_data.csv")
SCC_2014_2024 = SCC.query("year >= 2014")
SCC_2014_2024

,citation,dataset,year,language,document_date,source_url,unofficial_text,inadmissibility_reason
136,2014 SCC 16,SCC,2014,en,2014-02-21,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,R. v. Babos\nCollection\nSupreme Court Judgmen...,"['security', 'human_rights', 'serious_criminal..."
137,2014 SCC 24,SCC,2014,en,2014-03-27,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,Mission Institution v. Khela\nCollection\nSupr...,"['security', 'human_rights', 'serious_criminal..."
138,2014 SCC 37,SCC,2014,en,2014-05-14,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,Canada (Citizenship and Immigration) v. Harkat...,"['security', 'human_rights', 'serious_criminal..."
139,2014 SCC 62,SCC,2014,en,2014-10-10,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,Kazemi Estate v. Islamic Republic of Iran\nCol...,"['security', 'human_rights', 'serious_criminal..."
140,2014 SCC 67,SCC,2014,en,2014-10-28,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,Thibodeau v. Air Canada\nCollection\nSupreme C...,"['security', 'human_rights', 'misrepresentation']"
...,...,...,...,...,...,...,...,...
210,2024 SCC 37,SCC,2024,en,2024-11-08,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,TransAlta Generation Partnership v. Alberta\nC...,['human_rights']
211,2024 SCC 4,SCC,2024,en,2024-02-02,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,Ontario (Attorney General) v. Ontario (Informa...,"['security', 'human_rights', 'serious_criminal..."
212,2024 SCC 43,SCC,2024,en,2024-12-20,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,Quebec (Commission des droits de la personne e...,"['security', 'human_rights', 'serious_criminal..."
213,2024 SCC 8,SCC,2024,en,2024-03-15,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,Yatar v. TD Insurance Meloche Monnex\nCollecti...,"['serious_criminality', 'criminality', 'misrep..."


In [12]:
SSC_judges = extract_judges_from_dataframe(SCC_2014_2024, text_column="unofficial_text")

Row 136 processed.
Row 137 processed.
Row 138 processed.
Row 139 processed.
Row 140 processed.
Row 141 processed.
Row 142 processed.
Row 143 processed.
Row 144 processed.
Row 145 processed.
Row 146 processed.
Row 147 processed.
Row 148 processed.
Row 149 processed.
Row 150 processed.
Row 151 processed.
Row 152 processed.
Row 153 processed.
Row 154 processed.
Row 155 processed.
Row 156 processed.
Row 157 processed.
Row 158 processed.
Row 159 processed.
Row 160 processed.
Row 161 processed.
Row 162 processed.
Row 163 processed.
Row 164 processed.
Row 165 processed.
Row 166 processed.
Row 167 processed.
Row 168 processed.
Row 169 processed.
Row 170 processed.
Row 171 processed.
Row 172 processed.
Row 173 processed.
Row 174 processed.
Row 175 processed.
Row 176 processed.
Row 177 processed.
Row 178 processed.
Row 179 processed.
Row 180 processed.
Row 181 processed.
Row 182 processed.
Row 183 processed.
Row 184 processed.
Row 185 processed.
Row 186 processed.
Row 187 processed.
Row 188 proc

In [35]:
SSC_final = SSC_judges.query("~inadmissibility_reason.str.contains('other') and ~inadmissibility_reason.str.contains('refugee')", engine='python')
SSC_final

,citation,dataset,year,language,document_date,source_url,unofficial_text,inadmissibility_reason,judges
136,2014 SCC 16,SCC,2014,en,2014-02-21,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,R. v. Babos\nCollection\nSupreme Court Judgmen...,"['security', 'human_rights', 'serious_criminal...","[McLachlin, LeBel, Abella, Cromwell, Moldaver,..."
137,2014 SCC 24,SCC,2014,en,2014-03-27,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,Mission Institution v. Khela\nCollection\nSupr...,"['security', 'human_rights', 'serious_criminal...","[McLachlin, LeBel, Abella, Rothstein, Cromwell..."
138,2014 SCC 37,SCC,2014,en,2014-05-14,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,Canada (Citizenship and Immigration) v. Harkat...,"['security', 'human_rights', 'serious_criminal...","[McLachlin, LeBel, Abella, Rothstein, Cromwell..."
139,2014 SCC 62,SCC,2014,en,2014-10-10,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,Kazemi Estate v. Islamic Republic of Iran\nCol...,"['security', 'human_rights', 'serious_criminal...","[McLachlin, LeBel, Abella, Rothstein, Cromwell..."
140,2014 SCC 67,SCC,2014,en,2014-10-28,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,Thibodeau v. Air Canada\nCollection\nSupreme C...,"['security', 'human_rights', 'misrepresentation']","[McLachlin, LeBel, Abella, Rothstein, Cromwell..."
...,...,...,...,...,...,...,...,...,...
210,2024 SCC 37,SCC,2024,en,2024-11-08,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,TransAlta Generation Partnership v. Alberta\nC...,['human_rights'],"[Wagner, Karakatsanis, Côté, Rowe, Martin, Kas..."
211,2024 SCC 4,SCC,2024,en,2024-02-02,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,Ontario (Attorney General) v. Ontario (Informa...,"['security', 'human_rights', 'serious_criminal...",[]
212,2024 SCC 43,SCC,2024,en,2024-12-20,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,Quebec (Commission des droits de la personne e...,"['security', 'human_rights', 'serious_criminal...","[Wagner, Karakatsanis, Côté, Rowe, Martin, Kas..."
213,2024 SCC 8,SCC,2024,en,2024-03-15,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,Yatar v. TD Insurance Meloche Monnex\nCollecti...,"['serious_criminality', 'criminality', 'misrep...","[Wagner, Karakatsanis, Côté, Rowe, Martin, Kas..."


In [36]:
SSC_final.to_csv("../data/processed/SCC_judges.csv")

## Federal Court of Appeal

In [14]:
FCA = pd.read_csv("../data/processed/FCA_data.csv")
FCA_2014_2024 = FCA.query("year >= 2014")
FCA_2014_2024

,citation,dataset,year,language,document_date,source_url,unofficial_text,inadmissibility_reason
478,2014 FCA 11,FCA,2014,en,2014-01-21,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Bremsak v. Professional Institute of the Publi...,"['security', 'organized_criminality']"
479,2014 FCA 110,FCA,2014,en,2014-05-02,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Canada (Attorney General) v. Johnstone\nCourt ...,"['human_rights', 'serious_criminality', 'crimi..."
480,2014 FCA 113,FCA,2014,en,2014-05-02,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Kanthasamy v. Canada (Citizenship and Immigrat...,['refugee']
481,2014 FCA 114,FCA,2014,en,2014-05-02,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Lemus v. Canada (Citizenship and Immigration)\...,['refugee']
482,2014 FCA 126,FCA,2014,en,2014-05-14,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Kinsel v. Canada (Citizenship and Immigration)...,"['security', 'human_rights', 'serious_criminal..."
...,...,...,...,...,...,...,...,...
1039,2016 CAF 96,FCA,2016,fr,2016-03-29,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Canada (Citoyenneté et Immigration) c. Singh\n...,"['human_rights', 'serious_criminality', 'crimi..."
1040,2018 CAF 136,FCA,2018,fr,2018-07-18,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Bonnybrook Park Industrial Development Co. Ltd...,"['human_rights', 'serious_criminality', 'crimi..."
1041,2018 CAF 138,FCA,2018,fr,2018-07-19,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Revell c. Canada (Citoyenneté et Immigration)\...,"['serious_criminality', 'criminality']"
1042,2018 CAF 143,FCA,2018,fr,2018-07-26,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Bousaleh c. Canada (Citoyenneté et Immigration...,['refugee']


In [15]:
FCA_judges = extract_judges_from_dataframe(FCA_2014_2024, text_column="unofficial_text")

Row 478 processed.
Row 479 processed.
Row 480 processed.
Row 481 processed.
Row 482 processed.
Row 483 processed.
Row 484 processed.
Row 485 processed.
Row 486 processed.
Row 487 processed.
Row 488 processed.
Row 489 processed.
Row 490 processed.
Row 491 processed.
Row 492 processed.
Row 493 processed.
Row 494 processed.
Row 495 processed.
Row 496 processed.
Row 497 processed.
Row 498 processed.
Row 499 processed.
Row 500 processed.
Row 501 processed.
Row 502 processed.
Row 503 processed.
Row 504 processed.
Row 505 processed.
Row 506 processed.
Row 507 processed.
Row 508 processed.
Row 509 processed.
Row 510 processed.
Row 511 processed.
Row 512 processed.
Row 513 processed.
Row 514 processed.
Row 515 processed.
Row 516 processed.
Row 517 processed.
Row 518 processed.
Row 519 processed.
Row 520 processed.
Row 521 processed.
Row 522 processed.
Row 523 processed.
Row 524 processed.
Row 525 processed.
Row 526 processed.
Row 527 processed.
Row 528 processed.
Row 529 processed.
Row 530 proc

In [33]:
FCA_final = FCA_judges.query("~inadmissibility_reason.str.contains('other') and ~inadmissibility_reason.str.contains('refugee')", engine='python')
FCA_final

,citation,dataset,year,language,document_date,source_url,unofficial_text,inadmissibility_reason,judges
478,2014 FCA 11,FCA,2014,en,2014-01-21,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Bremsak v. Professional Institute of the Publi...,"['security', 'organized_criminality']","[DAWSON, TRUDEL, NEAR]"
479,2014 FCA 110,FCA,2014,en,2014-05-02,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Canada (Attorney General) v. Johnstone\nCourt ...,"['human_rights', 'serious_criminality', 'crimi...","[PELLETIER, MAINVILLE, SCOTT]"
482,2014 FCA 126,FCA,2014,en,2014-05-14,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Kinsel v. Canada (Citizenship and Immigration)...,"['security', 'human_rights', 'serious_criminal...","[Dawson, TruDEL, Near]"
483,2014 FCA 14,FCA,2014,en,2014-01-22,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Desgagnés Transarctik Inc. v. Canada (Attorney...,"['human_rights', 'serious_criminality']","[PELLETIER, GAUTHIER, TRUDEL]"
484,2014 FCA 157,FCA,2014,en,2014-06-10,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Sanchez v. Canada (Citizenship and Immigration...,"['serious_criminality', 'criminality']","[NADON, STRATAS, SCOTT]"
...,...,...,...,...,...,...,...,...,...
1023,2024 FCA 95,FCA,2024,en,2024-05-15,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Canada (Attorney General) v. Public Service Al...,"['human_rights', 'serious_criminality', 'crimi...","[Stratás, Mactavish, Walker]"
1039,2016 CAF 96,FCA,2016,fr,2016-03-29,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Canada (Citoyenneté et Immigration) c. Singh\n...,"['human_rights', 'serious_criminality', 'crimi...","[Nadon, Gauthier, De Montigny]"
1040,2018 CAF 136,FCA,2018,fr,2018-07-18,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Bonnybrook Park Industrial Development Co. Ltd...,"['human_rights', 'serious_criminality', 'crimi...","[Stratas, Near, Woods]"
1041,2018 CAF 138,FCA,2018,fr,2018-07-19,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Revell c. Canada (Citoyenneté et Immigration)\...,"['serious_criminality', 'criminality']",[GLEASON]


In [34]:
FCA_final.to_csv("../data/processed/FCA_judges.csv")

# Federal Court

In [17]:
FC = pd.read_csv("../data/processed/FC_data.csv")
FC_2014_2024 = FC.query("year >= 2014")
FC_2014_2024

,citation,dataset,year,language,document_date,source_url,unofficial_text,inadmissibility_reason
11922,2014 FC 1,FC,2014,en,2014-01-02,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Telus Communications Company v. Canada (Attorn...,"['human_rights', 'serious_criminality', 'crimi..."
11923,2014 FC 10,FC,2014,en,2014-01-07,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Singh v. Canada (Citizenship and Immigration)\...,['refugee']
11924,2014 FC 1000,FC,2014,en,2014-10-21,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Vinat v. Canada (Citizenship and Immigration)\...,['other']
11925,2014 FC 1002,FC,2014,en,2014-10-23,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Almrei v. Canada (Citizenship and Immigration)...,['refugee']
11926,2014 FC 1003,FC,2014,en,2014-10-22,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Avagyan v. Canada (Citizenship and Immigration...,['refugee']
...,...,...,...,...,...,...,...,...
23176,2023 CF 626,FC,2023,fr,2023-04-28,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Roudehchianahmadi c. Canada (Citoyenneté et Im...,['other']
23177,2024 CF 151,FC,2024,fr,2024-01-30,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Salazar Vargas c. Canada (Citoyenneté et Immig...,"['serious_criminality', 'criminality']"
23178,2024 CF 327,FC,2024,fr,2024-02-28,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Rashidian c. Canada (Procureur général)\nBase ...,['human_rights']
23179,2024 CF 42,FC,2024,fr,2024-01-23,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Canadian Frontline Nurses c. Canada (Procureur...,['refugee']


In [37]:
FC_final = FC_2014_2024.query("~inadmissibility_reason.str.contains('other') and ~inadmissibility_reason.str.contains('refugee')", engine='python')
FC_final

,citation,dataset,year,language,document_date,source_url,unofficial_text,inadmissibility_reason
11922,2014 FC 1,FC,2014,en,2014-01-02,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Telus Communications Company v. Canada (Attorn...,"['human_rights', 'serious_criminality', 'crimi..."
11930,2014 FC 1008,FC,2014,en,2014-10-22,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Bautista v. Canada (Citizenship and Immigratio...,['misrepresentation']
11933,2014 FC 1011,FC,2014,en,2014-10-23,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Canada (Public Safety and Emergency Preparedne...,['non_compliance']
11934,2014 FC 1012,FC,2014,en,2014-10-23,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Po v. Canada (Citizenship and Immigration)\nCo...,['human_rights']
11935,2014 FC 1015,FC,2014,en,2014-10-24,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Gordon v. Canada (Citizenship and Immigration)...,"['human_rights', 'misrepresentation']"
...,...,...,...,...,...,...,...,...
23170,2022 CF 882,FC,2022,fr,2022-06-16,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Jules c. Canada (Citoyenneté et Immigration)\n...,"['human_rights', 'serious_criminality', 'crimi..."
23173,2023 CF 1619,FC,2023,fr,2023-12-01,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Salhi c. Canada (Citoyenneté et Immigration)\n...,"['serious_criminality', 'criminality']"
23177,2024 CF 151,FC,2024,fr,2024-01-30,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Salazar Vargas c. Canada (Citoyenneté et Immig...,"['serious_criminality', 'criminality']"
23178,2024 CF 327,FC,2024,fr,2024-02-28,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Rashidian c. Canada (Procureur général)\nBase ...,['human_rights']


In [38]:
FC_judges = extract_judges_from_dataframe(FC_final, text_column="unofficial_text")

Row 11922 processed.
Row 11930 processed.
Row 11933 processed.
Row 11934 processed.
Row 11935 processed.
Row 11941 processed.
Row 11944 processed.
Row 11947 processed.
Row 11948 processed.
Row 11949 processed.
Row 11950 processed.
Row 11951 processed.
Row 11954 processed.
Row 11955 processed.
Row 11957 processed.
Row 11959 processed.
Row 11960 processed.
Row 11961 processed.
Row 11967 processed.
Row 11968 processed.
Row 11971 processed.
Row 11977 processed.
Row 11979 processed.
Row 11983 processed.
Row 11985 processed.
Row 11990 processed.
Row 11994 processed.
Row 11996 processed.
Row 11998 processed.
Row 12001 processed.
Row 12003 processed.
Row 12004 processed.
Row 12007 processed.
Row 12010 processed.
Row 12014 processed.
Row 12018 processed.
Row 12020 processed.
Row 12021 processed.
Row 12025 processed.
Row 12027 processed.
Row 12034 processed.
Row 12038 processed.
Row 12042 processed.
Row 12043 processed.
Row 12046 processed.
Row 12048 processed.
Row 12052 processed.
Row 12053 pro

In [39]:
FC_judges.to_csv("../data/processed/FC_judges.csv")

In [40]:
FC_judges

,citation,dataset,year,language,document_date,source_url,unofficial_text,inadmissibility_reason,judges
11922,2014 FC 1,FC,2014,en,2014-01-02,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Telus Communications Company v. Canada (Attorn...,"['human_rights', 'serious_criminality', 'crimi...",[Strickland]
11930,2014 FC 1008,FC,2014,en,2014-10-22,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Bautista v. Canada (Citizenship and Immigratio...,['misrepresentation'],[Diner]
11933,2014 FC 1011,FC,2014,en,2014-10-23,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Canada (Public Safety and Emergency Preparedne...,['non_compliance'],"[Shore, Gauthier]"
11934,2014 FC 1012,FC,2014,en,2014-10-23,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Po v. Canada (Citizenship and Immigration)\nCo...,['human_rights'],[Tremblay-Lamer]
11935,2014 FC 1015,FC,2014,en,2014-10-24,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Gordon v. Canada (Citizenship and Immigration)...,"['human_rights', 'misrepresentation']",[Locke]
...,...,...,...,...,...,...,...,...,...
23170,2022 CF 882,FC,2022,fr,2022-06-16,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Jules c. Canada (Citoyenneté et Immigration)\n...,"['human_rights', 'serious_criminality', 'crimi...",[St-Louis]
23173,2023 CF 1619,FC,2023,fr,2023-12-01,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Salhi c. Canada (Citoyenneté et Immigration)\n...,"['serious_criminality', 'criminality']",[Azmudeh]
23177,2024 CF 151,FC,2024,fr,2024-01-30,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Salazar Vargas c. Canada (Citoyenneté et Immig...,"['serious_criminality', 'criminality']",[Heneghan]
23178,2024 CF 327,FC,2024,fr,2024-02-28,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Rashidian c. Canada (Procureur général)\nBase ...,['human_rights'],"[Ngo, Tabib, Steele]"
